連接官方資料庫


讀取25筆資料

In [3]:
from neo4j import GraphDatabase
import os
from dotenv import load_dotenv

# 載入環境變量
load_dotenv()

# 連接信息
DEMO_URI = "neo4j+s://demo.neo4jlabs.com:7687"
DEMO_DATABASE = "movies"
DEMO_USER = "movies"
DEMO_PASSWORD = "movies"

# 連接資料庫
demo_driver = GraphDatabase.driver(DEMO_URI, auth=(DEMO_USER, DEMO_PASSWORD))

def fetch_data():
    # 從官方資料庫獲取數據
    query = """
    MATCH (n) RETURN n LIMIT 25
    """
    try:
        with demo_driver.session(database=DEMO_DATABASE) as session:
            results = session.run(query)
            nodes = [{"type": record["n"].labels, "properties": dict(record["n"].items())} for record in results]
            return nodes
    except Exception as e:
        print(f"An error occurred: {e}")
        return []

if __name__ == "__main__":
    nodes = fetch_data()
    if nodes:
        print(f"Retrieved {len(nodes)} nodes from the database.")
        # 輸出節點信息
        for node in nodes:
            print("Node Type:", node["type"], "| Properties:", node["properties"])
    else:
        print("No nodes retrieved.")


Retrieved 25 nodes from the database.
Node Type: frozenset({'Movie'}) | Properties: {'tagline': 'Welcome to the Real World', 'votes': 5254, 'title': 'The Matrix', 'released': 1999}
Node Type: frozenset({'Person'}) | Properties: {'born': 1964, 'name': 'Keanu Reeves'}
Node Type: frozenset({'Person'}) | Properties: {'born': 1967, 'name': 'Carrie-Anne Moss'}
Node Type: frozenset({'Person'}) | Properties: {'born': 1961, 'name': 'Laurence Fishburne'}
Node Type: frozenset({'Person'}) | Properties: {'born': 1960, 'name': 'Hugo Weaving'}
Node Type: frozenset({'Person'}) | Properties: {'born': 1967, 'name': 'Lilly Wachowski'}
Node Type: frozenset({'Person'}) | Properties: {'born': 1965, 'name': 'Lana Wachowski'}
Node Type: frozenset({'Person'}) | Properties: {'born': 1952, 'name': 'Joel Silver'}
Node Type: frozenset({'Person'}) | Properties: {'born': 1978, 'name': 'Emil Eifrem'}
Node Type: frozenset({'Movie'}) | Properties: {'tagline': 'Free your mind', 'votes': 1537, 'title': 'The Matrix Reload

讀取全部資料，並寫入自己的資料庫

In [14]:
from neo4j import GraphDatabase
import os
from dotenv import load_dotenv

# 載入環境變數
load_dotenv()

# 連接信息
DEMO_URI = "neo4j+s://demo.neo4jlabs.com:7687"
DEMO_DATABASE = "movies"
DEMO_USER = "movies"
DEMO_PASSWORD = "movies"

# 自己的資料庫連接信息
YOUR_URI = os.getenv("NEO4J_URI")
YOUR_USER = os.getenv("NEO4J_USER")
YOUR_PASSWORD = os.getenv("NEO4J_PASSWORD")

# 連接官方資料庫
demo_driver = GraphDatabase.driver(DEMO_URI, auth=(DEMO_USER, DEMO_PASSWORD))
# 連接自己的資料庫
your_driver = GraphDatabase.driver(YOUR_URI, auth=(YOUR_USER, YOUR_PASSWORD))

def fetch_data():
    # 從官方資料庫獲取節點及關係
    query = """
    MATCH (n)-[r]->(m) RETURN n, r, m
    """
    try:
        with demo_driver.session(database=DEMO_DATABASE) as session:
            results = session.run(query)
            data = []
            for record in results:
                data.append({
                    "start_node": {
                        "labels": list(record["n"].labels),
                        "properties": dict(record["n"].items())
                    },
                    "relationship": {
                        "type": record["r"].type,
                        "properties": dict(record["r"].items())
                    },
                    "end_node": {
                        "labels": list(record["m"].labels),
                        "properties": dict(record["m"].items())
                    }
                })
            return data
    except Exception as e:
        print(f"An error occurred: {e}")
        return []

def insert_data(data):
    # 將節點及關係數據插入自己的資料庫
    try:
        with your_driver.session() as session:
            for item in data:
                start_node_props = ', '.join(f'{k}: ${k}' for k in item['start_node']['properties'].keys())
                start_query = f"MERGE (n:{':'.join(item['start_node']['labels'])} {{{start_node_props}}}) RETURN id(n)"
                start_node_id = session.run(start_query, **item['start_node']['properties']).single()[0]
                
                end_node_props = ', '.join(f'{k}: ${k}' for k in item['end_node']['properties'].keys())
                end_query = f"MERGE (n:{':'.join(item['end_node']['labels'])} {{{end_node_props}}}) RETURN id(n)"
                end_node_id = session.run(end_query, **item['end_node']['properties']).single()[0]
                
                rel_props = ', '.join(f'{k}: ${k}' for k in item['relationship']['properties'].keys())
                rel_query = f"MATCH (a), (b) WHERE id(a) = $start_id AND id(b) = $end_id CREATE (a)-[r:{item['relationship']['type']} {{{rel_props}}}]->(b)"
                session.run(rel_query, start_id=start_node_id, end_id=end_node_id, **item['relationship']['properties'])

            print("Data and relationships have been inserted into your database.")
    except Exception as e:
        print(f"An error occurred while inserting data: {e}")

if __name__ == "__main__":
    data = fetch_data()
    if data:
        print(f"Retrieved {len(data)} items from the database.")
        insert_data(data)
    else:
        print("No data retrieved.")


Retrieved 253 items from the database.
Data and relationships have been inserted into your database.


透過語法查詢此時官方資料庫與自己資料庫的差異

In [4]:
from neo4j import GraphDatabase
import os
from dotenv import load_dotenv
import json

load_dotenv()

# 官方及自己的資料庫連接信息
DEMO_URI = "neo4j+s://demo.neo4jlabs.com:7687"
DEMO_DATABASE = "movies"
DEMO_USER = "movies"
DEMO_PASSWORD = "movies"
YOUR_URI = os.getenv("NEO4J_URI")
YOUR_USER = os.getenv("NEO4J_USER")
YOUR_PASSWORD = os.getenv("NEO4J_PASSWORD")

# 建立連接
demo_driver = GraphDatabase.driver(DEMO_URI, auth=(DEMO_USER, DEMO_PASSWORD))
your_driver = GraphDatabase.driver(YOUR_URI, auth=(YOUR_USER, YOUR_PASSWORD))

def fetch_data(driver, database):
    """從指定資料庫獲取所有節點及關係"""
    query_nodes = "MATCH (n) RETURN n"
    query_relationships = "MATCH ()-[r]->() RETURN r"
    data = {"nodes": [], "relationships": []}
    with driver.session(database=database) as session:
        # 获取节点
        nodes = session.run(query_nodes)
        for node in nodes:
            data["nodes"].append({"labels": list(node["n"].labels), "properties": dict(node["n"].items())})
        # 获取关系
        relationships = session.run(query_relationships)
        for rel in relationships:
            data["relationships"].append({"type": rel["r"].type, "properties": dict(rel["r"].items())})
    return data

def save_data(data, filename):
    """將資料保存為 JSON 文件"""
    with open(filename, 'w') as f:
        json.dump(data, f, indent=4)

if __name__ == "__main__":
    # 從兩個資料庫取得並保存數據
    demo_data = fetch_data(demo_driver, DEMO_DATABASE)
    save_data(demo_data, "demo_database_data.json")
    # 假設自己的資料庫沒有指定不同的資料庫上下文
    your_data = fetch_data(your_driver, None)
    save_data(your_data, "your_database_data.json")
    
    print("Data has been fetched and saved.")


Data has been fetched and saved.


優化腳本，進行排序：這個排序有瑕疵，需優化

In [6]:
from neo4j import GraphDatabase
import os
from dotenv import load_dotenv
import json

load_dotenv()

# 官方及自己的資料庫連接信息
DEMO_URI = "neo4j+s://demo.neo4jlabs.com:7687"
DEMO_DATABASE = "movies"
DEMO_USER = "movies"
DEMO_PASSWORD = "movies"
YOUR_URI = os.getenv("NEO4J_URI")
YOUR_USER = os.getenv("NEO4J_USER")
YOUR_PASSWORD = os.getenv("NEO4J_PASSWORD")

# 建立連接
demo_driver = GraphDatabase.driver(DEMO_URI, auth=(DEMO_USER, DEMO_PASSWORD))
your_driver = GraphDatabase.driver(YOUR_URI, auth=(YOUR_USER, YOUR_PASSWORD))

def fetch_and_sort_data(driver, database):
    """從指定資料庫取得所有節點和關係，並進行排序"""
    query_nodes = "MATCH (n) RETURN n"
    query_relationships = "MATCH ()-[r]->() RETURN r"
    data = {"nodes": [], "relationships": []}
    with driver.session(database=database) as session:
        # 獲取節點
        nodes = session.run(query_nodes)
        for node in nodes:
            data["nodes"].append({"labels": list(node["n"].labels), "properties": dict(node["n"].items())})
        data["nodes"].sort(key=lambda x: (x["labels"], x["properties"].get("name", "")))
        
        # 獲取關係
        relationships = session.run(query_relationships)
        for rel in relationships:
            data["relationships"].append({"type": rel["r"].type, "properties": dict(rel["r"].items())})
        data["relationships"].sort(key=lambda x: (x["type"], x["properties"].get("name", "")))
    
    return data

def save_data(data, filename):
    """將資料儲存為JSON文件"""
    with open(filename, 'w') as f:
        json.dump(data, f, indent=4)

if __name__ == "__main__":
    # 從兩個資料庫取得並保存數據
    demo_data = fetch_and_sort_data(demo_driver, DEMO_DATABASE)
    save_data(demo_data, "demo_database_data.json")
    # 假設自己的資料庫沒有指定不同的資料庫上下文
    your_data = fetch_and_sort_data(your_driver, None)
    save_data(your_data, "your_database_data.json")
    
    print("Data has been fetched and saved.")


Data has been fetched and saved.


優化排序腳本：添加排序函數

In [2]:
from neo4j import GraphDatabase
import os
from dotenv import load_dotenv
import json

load_dotenv()

# 官方和自己資料庫的連接訊息
DEMO_URI = "neo4j+s://demo.neo4jlabs.com:7687"
DEMO_DATABASE = "movies"
DEMO_USER = "movies"
DEMO_PASSWORD = "movies"
YOUR_URI = os.getenv("NEO4J_URI")
YOUR_USER = os.getenv("NEO4J_USER")
YOUR_PASSWORD = os.getenv("NEO4J_PASSWORD")

# 建立連線
demo_driver = GraphDatabase.driver(DEMO_URI, auth=(DEMO_USER, DEMO_PASSWORD))
your_driver = GraphDatabase.driver(YOUR_URI, auth=(YOUR_USER, YOUR_PASSWORD))

def fetch_and_sort_data(driver, database):
    """從指定資料庫取得所有節點和關係，並進行排序"""
    query_nodes = "MATCH (n) RETURN n"
    query_relationships = "MATCH ()-[r]->() RETURN r"
    data = {"nodes": [], "relationships": []}
    with driver.session(database=database) as session:
        # 獲取節點
        nodes = session.run(query_nodes)
        for node in nodes:
            data["nodes"].append({"labels": list(node["n"].labels), "properties": dict(node["n"].items())})
        
        # 獲取關係
        relationships = session.run(query_relationships)
        for rel in relationships:
            data["relationships"].append({"type": rel["r"].type, "properties": dict(rel["r"].items())})
    # 排序數據
    sort_data(data)
    return data

def sort_data(data):
    """對資料進行排序"""
    data["nodes"].sort(key=lambda x: (sorted(x["labels"]), sorted((k, v) for k, v in x["properties"].items())))
    data["relationships"].sort(key=lambda x: (x["type"], sorted((k, v) for k, v in x["properties"].items())))

def save_data(data, filename):
    """將資料儲存為JSON文件"""
    with open(filename, 'w') as f:
        json.dump(data, f, indent=4)

if __name__ == "__main__":
    # 從兩個資料庫中獲取並保存數據
    demo_data = fetch_and_sort_data(demo_driver, DEMO_DATABASE)
    save_data(demo_data, "demo_database_data.json")
    # 假設資料庫沒有指定不同的資料庫上下文
    your_data = fetch_and_sort_data(your_driver, None)
    save_data(your_data, "your_database_data.json")
    
    print("Data has been fetched and saved.")


Data has been fetched and saved.


比對兩個 JSON

In [5]:
import json

def compare_json(file1, file2):
    """比較兩個JSON文件並輸出不同之處，如果完全一致則輸出特定訊息"""
    with open(file1, 'r') as f1, open(file2, 'r') as f2:
        data1 = json.load(f1)
        data2 = json.load(f2)
    # 跟踪是否發現差異
    differences_found = False

    # 比對節點
    for item1, item2 in zip(data1["nodes"], data2["nodes"]):
        if item1 != item2:
            print(f"Difference in nodes: {item1} != {item2}")
            differences_found = True

    # 比對關係
    for item1, item2 in zip(data1["relationships"], data2["relationships"]):
        if item1 != item2:
            print(f"Difference in relationships: {item1} != {item2}")
            differences_found = True

    if not differences_found:
        print("完全一致")

if __name__ == "__main__":
    compare_json("demo_database_data.json", "your_database_data.json")


完全一致


抓取指定指令的數據

In [15]:
from neo4j import GraphDatabase
import os
from dotenv import load_dotenv
import json

# 官方和自己的資料庫連接訊息
DEMO_URI = "neo4j+s://demo.neo4jlabs.com:7687"
DEMO_DATABASE = "movies"
DEMO_USER = "movies"
DEMO_PASSWORD = "movies"
YOUR_DATABASE = "neo4j"
YOUR_URI = os.getenv("NEO4J_URI")
YOUR_USER = os.getenv("NEO4J_USER")
YOUR_PASSWORD = os.getenv("NEO4J_PASSWORD")



def fetch_and_save_data(uri, user, password, database, filename):
    driver = GraphDatabase.driver(uri, auth=(user, password))

    def fetch_wrote_relationships():
        """從資料庫獲取WROTE關係的數據"""
        query = "MATCH p=()-[r:WROTE]->() RETURN p"
        data = []
        with driver.session(database=database) as session:
            results = session.run(query)
            for record in results:
                path = record["p"]
                for relationship in path.relationships:
                    data.append({
                        "start_id": relationship.start_node.id,
                        "end_id": relationship.end_node.id,
                        "type": relationship.type,
                        "properties": dict(relationship.items())
                    })
        return data

    def save_data_to_json(data, file_name):
        """將數據保存到JSON文件"""
        with open(file_name, 'w') as file:
            json.dump(data, file, indent=4)
        print(f"Data has been fetched and saved to '{file_name}'.")

    # 執行抓取和保存
    wrote_relationships = fetch_wrote_relationships()
    save_data_to_json(wrote_relationships, filename)
    driver.close()

if __name__ == "__main__":

    # 儲存官方資料庫數據
    fetch_and_save_data(DEMO_URI, USER, PASSWORD, DEMO_DATABASE, "demo_wrote_relationships.json")

    # 儲存您的資料庫數據
    fetch_and_save_data(YOUR_URI, YOUR_USER, YOUR_PASSWORD, YOUR_DATABASE, "your_wrote_relationships.json")


/var/folders/vm/_zft15050r7cfh47y6v5z4k40000gn/T/ipykernel_54581/4237825336.py:31: DeprecationWarning: `id` is deprecated, use `element_id` instead
  "start_id": relationship.start_node.id,
/var/folders/vm/_zft15050r7cfh47y6v5z4k40000gn/T/ipykernel_54581/4237825336.py:32: DeprecationWarning: `id` is deprecated, use `element_id` instead
  "end_id": relationship.end_node.id,


Data has been fetched and saved to 'demo_wrote_relationships.json'.
Data has been fetched and saved to 'your_wrote_relationships.json'.
